<a href="https://colab.research.google.com/github/mohdadil93/mohdadil93/blob/main/Assignment4_MohamedAdil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [3]:
#import phi from train dataset
phi=np.loadtxt('/content/train.csv',dtype='float',delimiter=',',skiprows=1,usecols=tuple(range(1,14)))

In [4]:
#import phi_test from train dataset
phi_test=np.loadtxt('/content/test.csv',dtype='float',delimiter=',',skiprows=1,usecols=tuple(range(1,14)))

In [5]:
#import y from train dataset
y=np.loadtxt('/content/train.csv',dtype='float',delimiter=',',skiprows=1,usecols=14,ndmin=2)

In [6]:
#concatenate coloumn of 1s to right of phi and phi_test
phi=np.concatenate((phi,np.ones((400,1))),axis=1)
phi_test=np.concatenate((phi_test,np.ones((105,1))),axis=1)

In [7]:
#applying min max scaling on each coloumn of phi and phi_test
for x in range(0,13):
  c_max=max(phi[:,x])
  c_min=min(phi[:,x])
  phi[:,x]=(phi[:,x]-c_min)/(c_max-c_min)
  phi_test[:,x]=(phi_test[:,x]-c_min)/(c_max-c_min)

In [8]:
#applying log scaling on y
y=np.log(y)

In [12]:
#Function to calculate change in error function
def new_func(p,phi,w):
  if(p==2):
    new_func=(2*(np.dot(np.dot(np.transpose(phi),phi),w)-np.dot(np.transpose(phi),y))+lambd*p*np.power(np.absolute(w),(p-1)))
  if(p<2 and p>1):
    new_func=(2*(np.dot(np.dot(np.transpose(phi),phi),w)-np.dot(np.transpose(phi),y))+lambd*p*np.power(np.absolute(w),(p-1))*np.sign(w))
  return new_func

In [13]:
#Dictionary containing filenames as keys and p as values
filenames={
    'output.csv':2,
    'output_p1.csv':1.75,
    'output_p2.csv':1.5,
    'output_p3.csv':1.3
}

In [14]:
#For each item in this dictionary
for (fname,p) in filenames.items():
  #Set the w to all 0s
  w=np.zeros((14,1))

  lambd=0.2
  t=0.00012
  w_new=w-t*new_func(p,phi,w)

  i=0
  #Repeat steps until error between consecutive w is less than threshold
  while(np.linalg.norm(w_new-w)>10**-10):
    w=w_new
    w_new=w-t*new_func(p,phi,w)
    i=i+1

  #Load values of id
  id_test=np.loadtxt('/content/test.csv',dtype='int',delimiter=',',skiprows=1,usecols=0,ndmin=2)

  #Calculate y for test data
  y_test=np.exp(np.dot(phi_test,w_new))

  #Save the ids and y
  np.savetxt(fname,np.concatenate((id_test,y_test),axis=1),delimiter=',',fmt=['%d','%f'],header='ID,MEDV',comments='')